# Ames Housing Data Analysis.

## Problem Statement

This project examines a comprehensive housing dataset from the city of Ames in Iowa, USA (source). Homeowners looking to increase the value of their homes often spend too much remodelling and don't get the return on investment when selling the house. On the other hand, people looking to buy houses want to get the best house possible given a budget. The goal of this project is to address some of these concerns, specifically:

- what features add the most value to a home, and which hurt home values the most? given a set of features, 
- what is the expected sale price of a house?
- given a budget, what kind of house would one be able to afford?

To answer these questions, I fitted different linear regression models to the housing data in order to determine the features that are most influential on house price, and those that have the least effect. The performance of the models will be evaluated using the R2 metric, which is a measure of how much the model is able to explain the variance in the dataset.

 ([source](https://www.kaggle.com/c/dsi-us-6-project-2-regression-challenge/overview)).

## Executive Summary 

An elastic net regression model had the best predictive performance on housing sale price in Ames USA though we shall utilise a linar regression model for predictions; and outperformed the other linear models tested. The model revealed that square feet area, condition, age, and the location of the house are the most important determinant factors of how much a house sells for.

For house buyers with specific budgets, this model would allow one to figure out what features they would be able to afford given an amount of money. For those looking to invest, houses in the Northridge Heights, Stone Brook, and Northridge neighbourhoods all had higher prices compared to other neighbourhoods. Conversely, people looking to sell their house would be able to use the model to get an estimate of how much they would be able to sell their house for. If one is looking to sell, they should do it sooner rather than later, as the age of the house is the biggest contributing factor to the decrease in value. Having a garage in bad condition also negatively affect value, as does having a second floor to the house. And unsurprisingly, if the house is severely damaged, its value drops. As such, those looking to increase the value of their home could consider increasing the square footage of their house, renovating the kitchen and garage, and adding a fireplace.

The model may however have limited applicabilities, as it was developed using data on houses sold between 2006 - 2010 in Ames, USA. This dataset is limited in scope both in terms of the time frame captured, as well as location. The small time frame of four years is not enough to capture any annual changes in sale price that could arise as a result of external factors, such as the current economy. Housing prices at present (in 2019) may also have changed as a result of inflation. The model is also specific to houses in Ames and may not be as accurate when applied to data from another city or country, where house prices may be affected by different things than for example, garage quality.

In reality, house price may be difficult to predict as it is also affected by buyers' psychology, the economic climate, and other factors not included in the present dataset. There will never be a perfect model; the aim of this model is therefore not to give a perfect prediction, but act as a guideline to inform decisions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV, ElasticNet, ElasticNetCV 
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor

sns.set_style()

pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
#import all csv files for review. 
ah_df=pd.read_csv('~/desktop/DsI/submissions/projects/project_2/datasets/ameshousing.csv')
ssr= pd.read_csv('~/desktop/DsI/submissions/projects/project_2/datasets/sample_sub_reg.csv')
test = pd.read_csv('~/desktop/DsI/submissions/projects/project_2/datasets/test.csv')
train= pd.read_csv('~/desktop/DsI/submissions/projects/project_2/datasets/train.csv')

## Data cleaning

In [3]:
#congolomerate function defined to clean any dataframe from faults. assistance from codementors.com
def data_summaries (df, pred=None): 
    obs = df.shape[0]
    types = df.dtypes
    counts = df.apply(lambda x: x.count())
    uniques = df.apply(lambda x: [x.unique()])
    nulls = df.apply(lambda x: x.isnull().sum())
    distincts = df.apply(lambda x: x.unique().shape[0])
    missing_ration = (df.isnull().sum()/ obs) * 100
    skewness = df.skew() 
    print('Data shape:', df.shape)
    if prediction is None:
        columns = ['types', 'counts', 'distincts', 'nulls', 'missing ration', 'uniques']
        str = pd.concat([types, counts, distincts, nulls, missing_ration, uniques], axis = 1)

    else:
        corr = df.corr()[prediction]
        str = pd.concat([types, counts, distincts, nulls, missing_ration, uniques, corr], axis = 1, sort=False)
        corr_col = 'corr '  + prediction
        columns = ['types', 'counts', 'distincts', 'nulls', 'missing_ration', 'uniques', corr_col ]
    
    str.columns = cols
    dtypes = str.types.value_counts()
    print('___________________________\nData types:\n',str.types.value_counts())
    print('___________________________')
    return str

## Data Cleaning of Training & Testing. 

In [4]:
#Training DATA
#format acquired through research upon stackoverflow.com and tudored assistance upon codeMentor
def preprocess_train(input_df):
    # make copy of df
    df = input_df.copy()
    
    # make columns lowercase and remove spaces
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '')

    # function to drop columns
    def dropcol(df, cols_to_drop):
        df.drop(cols_to_drop, axis = 1, inplace = True)
    
    # 1 if feature present and 0 if feature absent
    pres = lambda x: 1 if x > 0 else 0
#add nascent columns

    # poRch 
    col_porch = ['3ssnporch','enclosedporch','openporchsf','screenporch']
    df['porchpres'] = df[col_porch].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
    dropcol(df, col_porch)
    
    # pooL
    col_pool = ['poolqc','poolarea']
    df['poolpres'] = df['poolarea'].apply(pres)
    dropcol(df, col_pool)
    
    # gaRagE
    df['garagepres'] = df['garagearea'].apply(pres)
    
    # FIREpLacE
    df['fireplacepres'] = df['fireplaces'].apply(pres)
    
    # MasoNRY VENEER
    df['masvnrpres'] = df['masvnrarea'].apply(pres)
    
    # basEMENT
    df['bsmtpres'] = df['bsmtqual'].fillna('none').apply(lambda x: 0 if x == 'none' else 1)

    # LaNDsLopE
    df['landslope'] = df['landslope'].apply(lambda x: 'flat' if x == 'gtl' else 'sloped')
    
    # MIscFEaTUREs
    df['shedpres'] = df['miscfeature'].apply(lambda x: 1 if x == 'shed' else 0)
    dropcol(df, 'miscfeature')
    
    # EXTERIoR MaTERIaLs
    exterior_materials = {'Metalsd': 'Metal',
                          'hdboard': 'Wood',
                          'Wd sdng': 'Wood',
                          'plywood': 'Wood',
                          'Wdshing': 'Wood',
                          'cemntbd': 'aggregate',
                          'stucco': 'aggregate',
                          'cblock': 'aggregate',
                          'asphshn': 'aggregate',
                          'Imstucc': 'aggregate',
                          'brkFace': 'brick',
                          'brkcomm': 'brick',
                          'asbshng': 'aesbestos',
                          'stone': 'stone',
                          'Vinylsd': 'Vinyl'}
    df['exterior'] = df['exterior1st'].map(exterior_materials)
    dropcol(df, ['exterior1st','exterior2nd'])

    # agE soLD
    for index, val in enumerate(df['yearbuilt']):
        if val == df.loc[index, 'yrsold']:
            df.loc[index, 'age_sold'] = 0
        else:
            df.loc[index, 'age_sold'] = df.loc[index,'yrsold'] - val
    
    # REMoDELLED
    for index, val in enumerate(df['yearremod/add']):
        if val == df.loc[index,'yearbuilt']:
            df.loc[index,'remodelled'] = 0
        else:
            df.loc[index,'remodelled'] = 1
    dropcol(df,['yearremod/add'])  
    
    # LIVINg aREa
    df['2ndfloorpres'] = df['2ndflrsf'].apply(pres)     
    # get sum of 1stflr and 2ndflr area
    df['flrsf'] = df['1stflrsf']+df['2ndflrsf']
    # create bighouse column
    for index, val in enumerate(df['flrsf']):
        if val == df.loc[index, 'grlivarea']:
            df.loc[index, 'bighouse'] = 0
        else:
            df.loc[index, 'bighouse'] = 1
    dropcol(df, ['flrsf','1stflrsf','2ndflrsf'])
            
#imputaton of null values 

    # LoT FRoNTagE
    # fill nan with neighbourhood median
    median_lotfrt_per_nbhood = df["lotfrontage"].groupby(df["neighborhood"])
    for nbhood, grouped in median_lotfrt_per_nbhood:
        index = (df['neighborhood'] == nbhood) & (df['lotfrontage'].isnull())
        df.loc[index,'lotfrontage'] = grouped.median()
    # for the few remaining rows from the neighborhoods without a median (i.e. if n < 3),
    # fill with overall lotfrontage median
    df['lotfrontage'] = df['lotfrontage'].fillna(df['lotfrontage'].median())

    # MasoNRY VENEER
    # the NaN values in masvnrarea and masvnrtype are likely houses without masonry veneers
    df['masvnrarea'] = df['masvnrarea'].fillna(0.0)
    df['masvnrtype'] = df['masvnrtype'].fillna('None')
    
    # basEMENT pRopERTIEs
    # NaN values will be replaced with 'None' (no basement present)
    for col in ['bsmtqual','bsmtexposure','bsmtfintype1']:
        df[col] = df[col].fillna('None')
    # for the NaNs in the numerical columns, they will be replaced with 0 (since no basement)
    for col in ['bsmtfullbath','bsmtfinsf1','bsmtunfsf']:
        df[col] = df[col].fillna(0.0)

    # FIREpLacE
    # null value likely represent no fireplace
    df['fireplacequ'] = df['fireplacequ'].fillna('None')
    
    # gaRagE
    # null values likely represent no garage
    for col in ['garagetype','garagefinish','garagecond']:
        df[col] = df[col].fillna('Na')
    for col in ['garagearea','garagecars','garagearea']:
        df[col] = df[col].fillna(0.0)
# column conversion(ordinal to numerical)
    
    # function to map new values
    def map_new_vals(colname,dictionary):
        df[colname] = df[colname].map(dictionary)
    
#defining dictionaries for renaming
    lotshape_di = {'Reg': 0,
                   'IR1': 1,
                   'IR2': 2,
                   'IR3': 3}
    bsmtex_di = {'None': 0,
                'No': 1,
                'Mn': 2,
                'av': 3,
                'gd': 4}
    qual_di = {'Ex': 5,
               'gd': 4,
               'Ta': 3,
               'Fa': 2,
               'po': 1,
               'None': 0}
#map new values
    map_new_vals('lotshape', dictionary = lotshape_di)
    map_new_vals('bsmtexposure', dictionary = bsmtex_di)
    map_new_vals('exterqual', dictionary = qual_di)
    map_new_vals('bsmtqual', dictionary = qual_di)
    map_new_vals('heatingqc', dictionary = qual_di)
    map_new_vals('kitchenqual', dictionary = qual_di)
    map_new_vals('fireplacequ', dictionary = qual_di)

#column conversion (numerical to categorical)
    
    df['mssubclass'] = df['mssubclass'].astype(str)

#-DRop coLUMNs-
        
# sKEWED coLUMNs
    col_to_drop = ['alley','miscval','lowqualfinsf','street','utilities','condition2','roofmatl',\
                   'heating','centralair','electrical','paveddrive','fence','saletype','bsmthalfbath',\
                   'bsmtfintype2','bsmtfinsf2','bsmtcond','extercond','garagequal']
    dropcol(df, col_to_drop)
# pID coLUMN
    dropcol(df, ['pid'])
#coLLINEaR coLUMNs
    col_collinear = ['garagecars','totrmsabvgrd', 'totalbsmtsf','garageyrblt']
    dropcol(df, col_collinear) 

#-REMoVE oUTLIERs-
    
    df.drop(df[df['grlivarea'] > 4_500].index, inplace = True)
    df.drop(df[df['lotfrontage'] > 300].index, inplace = True)
    df.drop(df[df['lotarea'] > 100_000].index, inplace = True)

#-cLEaNINg-
    
    # convert all Na to None
    df.replace('Na','None',inplace=True)

    # reset index
    df = df.reset_index()

    return df

In [5]:
#Testing DATA
#format from above line reformatted for testing dataset
def preprocess_test(input_df):
    # make copy of df
    df = input_df.copy()
    
    # make columns lowercase and remove spaces
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '')

    # function to drop columns
    def dropcol(df, cols_to_drop):
        df.drop(cols_to_drop, axis = 1, inplace = True)

#add nascent columns
    
    # 1 if feature present and 0 if feature absent
    pres = lambda x: 1 if x > 0 else 0
    
    # poRch 
    col_porch = ['3ssnporch','enclosedporch','openporchsf','screenporch']
    df['porchpres'] = df[col_porch].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
    dropcol(df, col_porch)
    
    # pooL
    col_pool = ['poolqc','poolarea']
    df['poolpres'] = df['poolarea'].apply(pres)
    dropcol(df, col_pool)
    
    # gaRagE
    df['garagepres'] = df['garagearea'].apply(pres)
    
    # FIREpLacE
    df['fireplacepres'] = df['fireplaces'].apply(pres)
    
    # MasoNRY VENEER
    df['masvnrpres'] = df['masvnrarea'].apply(pres)
    
    # basEMENT
    df['bsmtpres'] = df['bsmtqual'].fillna('none').apply(lambda x: 0 if x == 'none' else 1)

    # LaNDsLopE
    df['landslope'] = df['landslope'].apply(lambda x: 'flat' if x == 'gtl' else 'sloped')
    
    # MIscFEaTUREs
    df['shedpres'] = df['miscfeature'].apply(lambda x: 1 if x == 'shed' else 0)
    dropcol(df, 'miscfeature')
    
    # EXTERIoR MaTERIaLs
    exterior_materials = {'Metalsd': 'Metal',
                          'hdboard': 'Wood',
                          'Wd sdng': 'Wood',
                          'plywood': 'Wood',
                          'Wdshing': 'Wood',
                          'cemntbd': 'aggregate',
                          'stucco': 'aggregate',
                          'cblock': 'aggregate',
                          'asphshn': 'aggregate',
                          'Imstucc': 'aggregate',
                          'brkFace': 'brick',
                          'brkcomm': 'brick',
                          'asbshng': 'aesbestos',
                          'stone': 'stone',
                          'Vinylsd': 'Vinyl'}
    df['exterior'] = df['exterior1st'].map(exterior_materials)
    dropcol(df, ['exterior1st','exterior2nd'])

    # agE soLD
    for index, val in enumerate(df['yearbuilt']):
        if val == df.loc[index, 'yrsold']:
            df.loc[index, 'age_sold'] = 0
        else:
            df.loc[index, 'age_sold'] = df.loc[index,'yrsold'] - val
    
    # REMoDELLED
    for index, val in enumerate(df['yearremod/add']):
        if val == df.loc[index,'yearbuilt']:
            df.loc[index,'remodelled'] = 0
        else:
            df.loc[index,'remodelled'] = 1
    dropcol(df,['yearremod/add'])  
    
    # LIVINg aREa
    df['2ndfloorpres'] = df['2ndflrsf'].apply(pres)     
    df['flrsf'] = df['1stflrsf']+df['2ndflrsf']
    # create bighouse column
    for index, val in enumerate(df['flrsf']):
        if val == df.loc[index, 'grlivarea']:
            df.loc[index, 'bighouse'] = 0
        else:
            df.loc[index, 'bighouse'] = 1
    dropcol(df, ['flrsf','1stflrsf','2ndflrsf'])
            
    
    # LoT FRoNTagE
    median_lotfrt_per_nbhood = df["lotfrontage"].groupby(df["neighborhood"])
    for nbhood, grouped in median_lotfrt_per_nbhood:
        index = (df['neighborhood'] == nbhood) & (df['lotfrontage'].isnull())
        df.loc[index,'lotfrontage'] = grouped.median()
        
    df['lotfrontage'] = df['lotfrontage'].fillna(df['lotfrontage'].median())

    # MasoNRY VENEER
    df['masvnrarea'] = df['masvnrarea'].fillna(0.0)
    df['masvnrtype'] = df['masvnrtype'].fillna('None')
    
    # basEMENT pRopERTIEs
    for col in ['bsmtqual','bsmtexposure','bsmtfintype1']:
        df[col] = df[col].fillna('None')
    for col in ['bsmtfullbath','bsmtfinsf1','bsmtunfsf']:
        df[col] = df[col].fillna(0.0)

    # FIREpLacE
    df['fireplacequ'] = df['fireplacequ'].fillna('None')
    
    # gaRagE
    for col in ['garagetype','garagefinish','garagecond']:
        df[col] = df[col].fillna('Na')
    for col in ['garagearea','garagecars','garagearea']:
        df[col] = df[col].fillna(0.0)

#ordinal to numerical categorical conversion
    def map_new_vals(colname,dictionary):
        df[colname] = df[colname].map(dictionary)
    lotshape_di = {'Reg': 0,
                   'IR1': 1,
                   'IR2': 2,
                   'IR3': 3}
    bsmtex_di = {'None': 0,
                'No': 1,
                'Mn': 2,
                'av': 3,
                'gd': 4}
    qual_di = {'Ex': 5,
               'gd': 4,
               'Ta': 3,
               'Fa': 2,
               'po': 1,
               'None': 0}
    map_new_vals('lotshape', dictionary = lotshape_di)
    map_new_vals('bsmtexposure', dictionary = bsmtex_di)
    map_new_vals('exterqual', dictionary = qual_di)
    map_new_vals('bsmtqual', dictionary = qual_di)
    map_new_vals('heatingqc', dictionary = qual_di)
    map_new_vals('kitchenqual', dictionary = qual_di)
    map_new_vals('fireplacequ', dictionary = qual_di)

#inverse conversion(numerical to categorical)
    
    df['mssubclass'] = df['mssubclass'].astype(str)

#column Exclusion
        
    # sKEWED coLUMNs
    col_to_drop = ['alley','miscval','lowqualfinsf','street','utilities','condition2','roofmatl',\
                   'heating','centralair','electrical','paveddrive','fence','saletype','bsmthalfbath',\
                   'bsmtfintype2','bsmtfinsf2','bsmtcond','extercond','garagequal']
    dropcol(df, col_to_drop)
    
    # pID coLUMN
    dropcol(df, ['pid'])

    # coLLINEaR coLUMNs
    col_collinear = ['garagecars','totrmsabvgrd', 'totalbsmtsf','garageyrblt']
    dropcol(df, col_collinear) 

#outlier exclusion
#     df.drop(df[df['grlivarea'] > 4_500].index, inplace = True)
#     df.drop(df[df['lotfrontage'] > 300].index, inplace = True)
#     df.drop(df[df['lotarea'] > 100_000].index, inplace = True)

#cleaning missing errors. 
    
    # convert all Na to None
    df.replace('Na','None',inplace=True)

    # reset index
    df = df.reset_index()

    return df

In [6]:
#function in order to guage for str columns, print count of unique values for numerical columns, print % of zeros.
def data_summaries (df):
    for col in df.columns:
        print ("------ %s ------" % col)
        if df[col].dtype == np.object:
            print (df[col].value_counts())
        elif len(df[df[col] == 0]) >= 1026:
            print (col, '\n', '% zeros (hIgh!): ', round((len(df[df[col] == 0])/2051)*100),\
                   '\n', 'no. zeros: ',len(df[df[col]==0]))
        else:
            print (col, '\n', '% zeros: ', round((len(df[df[col] == 0])/2051)*100))
        print ('\n')
    
data_summaries(train)

------ Id ------
Id 
 % zeros:  0


------ PID ------
PID 
 % zeros:  0


------ MS SubClass ------
MS SubClass 
 % zeros:  0


------ MS Zoning ------
RL         1598
RM          316
FV          101
C (all)      19
RH           14
A (agr)       2
I (all)       1
Name: MS Zoning, dtype: int64


------ Lot Frontage ------
Lot Frontage 
 % zeros:  0


------ Lot Area ------
Lot Area 
 % zeros:  0


------ Street ------
Pave    2044
Grvl       7
Name: Street, dtype: int64


------ Alley ------
Grvl    85
Pave    55
Name: Alley, dtype: int64


------ Lot Shape ------
Reg    1295
IR1     692
IR2      55
IR3       9
Name: Lot Shape, dtype: int64


------ Land Contour ------
Lvl    1843
HLS      85
Bnk      80
Low      43
Name: Land Contour, dtype: int64


------ Utilities ------
AllPub    2049
NoSewr       1
NoSeWa       1
Name: Utilities, dtype: int64


------ Lot Config ------
Inside     1503
Corner      348
CulDSac     131
FR2          60
FR3           9
Name: Lot Config, dtype: int64


-

In [7]:
ah_df.columns = ah_df.columns.str.replace(' ', '')
ssr.columns = ssr.columns.str.replace(' ', '')
train.columns = train.columns.str.replace(' ', '')
train.columns = train.columns.str.lower()
test.columns = test.columns.str.replace(' ', '')
test.columns = test.columns.str.lower()

print(train.columns)
print(train.info())
train.sample(5)

Index(['id', 'pid', 'mssubclass', 'mszoning', 'lotfrontage', 'lotarea',
       'street', 'alley', 'lotshape', 'landcontour', 'utilities', 'lotconfig',
       'landslope', 'neighborhood', 'condition1', 'condition2', 'bldgtype',
       'housestyle', 'overallqual', 'overallcond', 'yearbuilt',
       'yearremod/add', 'roofstyle', 'roofmatl', 'exterior1st', 'exterior2nd',
       'masvnrtype', 'masvnrarea', 'exterqual', 'extercond', 'foundation',
       'bsmtqual', 'bsmtcond', 'bsmtexposure', 'bsmtfintype1', 'bsmtfinsf1',
       'bsmtfintype2', 'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 'heating',
       'heatingqc', 'centralair', 'electrical', '1stflrsf', '2ndflrsf',
       'lowqualfinsf', 'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath',
       'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'kitchenqual',
       'totrmsabvgrd', 'functional', 'fireplaces', 'fireplacequ', 'garagetype',
       'garageyrblt', 'garagefinish', 'garagecars', 'garagearea', 'garagequal',
       'garagecond', 'pa

,id,pid,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremod/add,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,saleprice
1833,684,535453100,80,RL,80.0,8000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,SLvl,5,5,1959,1959,Gable,CompShg,BrkFace,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,Av,GLQ,433.0,Rec,95.0,0.0,528.0,GasA,TA,Y,SBrkr,1183,0,0,1183,1.0,0.0,1,0,3,1,TA,6,Typ,0,NaN,Attchd,1959.0,RFn,1.0,288.0,TA,TA,Y,0,0,0,0,0,0,NaN,GdWo,NaN,0,7,2009,WD,138000
2013,112,534151175,60,RL,80.0,10791,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,6,5,1993,1993,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,PConc,Gd,TA,Mn,GLQ,1137.0,Unf,0.0,143.0,1280.0,GasA,Ex,Y,SBrkr,1280,1215,0,2495,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,1993.0,Unf,2.0,660.0,TA,TA,Y,224,32,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,243500
311,2541,534250300,60,RL,NaN,14803,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,NWAmes,Norm,Norm,1Fam,2Story,6,5,1971,1971,Gable,CompShg,HdBoard,HdBoard,BrkFace,252.0,TA,TA,CBlock,TA,TA,No,Rec,416.0,Unf,0.0,409.0,825.0,GasA,Gd,Y,SBrkr,1097,896,0,1993,0.0,0.0,2,1,4,1,TA,8,Typ,1,Gd,Attchd,1971.0,RFn,2.0,495.0,TA,TA,Y,0,66,0,0,0,0,NaN,GdWo,NaN,0,6,2006,WD,190000
1270,1569,914476080,90,RL,76.0,10260,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,Duplex,2Story,5,4,1976,1976,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,GasA,TA,Y,SBrkr,936,936,0,1872,0.0,0.0,2,2,4,2,TA,8,Typ,0,NaN,Attchd,1976.0,Unf,2.0,484.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,11,2008,WD,100000
1997,2780,907194130,60,RL,65.0,14006,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2002,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,144.0,Gd,TA,PConc,Gd,TA,NaN,Unf,0.0,Unf,0.0,936.0,936.0,GasA,Ex,Y,SBrkr,936,840,0,1776,0.0,0.0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,2002.0,RFn,2.0,474.0,TA,TA,Y,144,96,0,0,0,0,NaN,NaN,NaN,0,2,2006,WD,192500


In [8]:
#removing duplicates from all dataframes
ah_df.drop_duplicates(subset=None, keep='first', inplace=True)
test.drop_duplicates(subset=None, keep='first', inplace=True)
train.drop_duplicates(subset=None, keep='first', inplace=True)

In [9]:
#verifying the count of nulls within any column
nulldf = pd.DataFrame(train.isnull().sum(), columns = ['nullcount'])
nulldf = nulldf[nulldf['nullcount'] != 0]
print(nulldf.shape)
nulldf

(26, 1)


,nullcount
lotfrontage,330
alley,1911
masvnrtype,22
masvnrarea,22
bsmtqual,55
bsmtcond,55
bsmtexposure,58
bsmtfintype1,55
bsmtfinsf1,1
bsmtfintype2,56


In [10]:
train.alley.value_counts()

Grvl    85
Pave    55
Name: alley, dtype: int64

In [11]:
train.poolqc.value_counts() 

Gd    4
Fa    2
TA    2
Ex    1
Name: poolqc, dtype: int64

In [12]:
train.garagequal.value_counts()

TA    1832
Fa      82
Gd      18
Ex       3
Po       2
Name: garagequal, dtype: int64

In [13]:
train.fireplacequ.value_counts() 

Gd    523
TA    407
Fa     59
Po     31
Ex     31
Name: fireplacequ, dtype: int64

In [14]:
train.bsmtqual.value_counts()

TA    887
Gd    864
Ex    184
Fa     60
Po      1
Name: bsmtqual, dtype: int64

In [15]:
print(train[train[['masvnrtype','masvnrtype']].isnull().any(axis=1)][['masvnrtype','masvnrtype']].shape)
train[train[['masvnrtype','masvnrtype']].isnull().any(axis=1)][['masvnrtype','masvnrtype']]

(22, 2)


,masvnrtype,masvnrtype
22,NaN,NaN
41,NaN,NaN
86,NaN,NaN
212,NaN,NaN
276,NaN,NaN
338,NaN,NaN
431,NaN,NaN
451,NaN,NaN
591,NaN,NaN
844,NaN,NaN


In [16]:
print(test[test[['masvnrtype','masvnrtype']].isnull().any(axis=1)][['masvnrtype','masvnrtype']].shape)
test[test[['masvnrtype','masvnrtype']].isnull().any(axis=1)][['masvnrtype','masvnrtype']]

(1, 2)


,masvnrtype,masvnrtype
865,NaN,NaN


In [17]:
#transpositon of train data
display(train.describe().transpose())

,count,mean,std,min,25%,50%,75%,max
id,2051.0,1.474034e+03,8.439808e+02,1.0,753.5,1486.0,2.198000e+03,2930.0
pid,2051.0,7.135900e+08,1.886918e+08,526301100.0,528458140.0,535453200.0,9.071801e+08,924152030.0
mssubclass,2051.0,5.700878e+01,4.282422e+01,20.0,20.0,50.0,7.000000e+01,190.0
lotfrontage,1721.0,6.905520e+01,2.326065e+01,21.0,58.0,68.0,8.000000e+01,313.0
lotarea,2051.0,1.006521e+04,6.742489e+03,1300.0,7500.0,9430.0,1.151350e+04,159000.0
overallqual,2051.0,6.112140e+00,1.426271e+00,1.0,5.0,6.0,7.000000e+00,10.0
overallcond,2051.0,5.562165e+00,1.104497e+00,1.0,5.0,5.0,6.000000e+00,9.0
yearbuilt,2051.0,1.971709e+03,3.017789e+01,1872.0,1953.5,1974.0,2.001000e+03,2010.0
yearremod/add,2051.0,1.984190e+03,2.103625e+01,1950.0,1964.5,1993.0,2.004000e+03,2010.0
masvnrarea,2029.0,9.969591e+01,1.749631e+02,0.0,0.0,0.0,1.610000e+02,1600.0


In [18]:
#transpositon of test data
display(test.describe().transpose())

,count,mean,std,min,25%,50%,75%,max
id,878.0,1.444749e+03,8.508383e+02,2.0,6.922500e+02,1433.0,2.197500e+03,2.928000e+03
pid,878.0,7.162867e+08,1.889103e+08,526302110.0,5.284841e+08,535454155.0,9.071904e+08,1.007100e+09
mssubclass,878.0,5.826879e+01,4.223541e+01,20.0,2.000000e+01,50.0,7.000000e+01,1.900000e+02
lotfrontage,718.0,6.954596e+01,2.353394e+01,21.0,5.900000e+01,68.0,8.000000e+01,1.820000e+02
lotarea,878.0,1.030703e+04,1.000267e+04,1477.0,7.297250e+03,9446.0,1.158900e+04,2.152450e+05
overallqual,878.0,6.050114e+00,1.369065e+00,2.0,5.000000e+00,6.0,7.000000e+00,1.000000e+01
overallcond,878.0,5.566059e+00,1.128903e+00,1.0,5.000000e+00,5.0,6.000000e+00,9.000000e+00
yearbuilt,878.0,1.970492e+03,3.039589e+01,1880.0,1.954000e+03,1972.0,1.999750e+03,2.010000e+03
yearremod/add,878.0,1.984418e+03,2.045072e+01,1950.0,1.967000e+03,1992.0,2.003000e+03,2.010000e+03
masvnrarea,877.0,1.062360e+02,1.871589e+02,0.0,0.000000e+00,0.0,1.720000e+02,1.378000e+03


In [19]:
#not entirely notable column, shall exclude later
train[train['garagecars'].isnull()]

,id,pid,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremod/add,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,saleprice
1712,2237,910201180,70,RM,50.0,9060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,2Story,5,6,1923,1999,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,BrkTil,Gd,TA,No,ALQ,548.0,Unf,0.0,311.0,859.0,GasA,Ex,Y,SBrkr,942,886,0,1828,0.0,0.0,2,0,3,1,Gd,6,Typ,0,NaN,Detchd,NaN,NaN,NaN,NaN,NaN,NaN,Y,174,0,212,0,0,0,NaN,MnPrv,NaN,0,3,2007,WD,150909


In [20]:
#not noteable of a column to include based upon null values
train[train['bsmtfullbath'].isnull()]

,id,pid,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremod/add,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,saleprice
616,1498,908154080,20,RL,123.0,47007,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,5,7,1959,1996,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,GasA,TA,Y,SBrkr,3820,0,0,3820,NaN,NaN,3,1,5,1,Ex,11,Typ,2,Gd,Attchd,1959.0,Unf,2.0,624.0,TA,TA,Y,0,372,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,284700
1327,1342,903230120,20,RM,99.0,5940,Pave,NaN,IR1,Lvl,AllPub,FR3,Gtl,BrkSide,Feedr,Norm,1Fam,1Story,4,7,1946,1950,Gable,CompShg,MetalSd,CBlock,None,0.0,TA,TA,PConc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GasA,TA,Y,FuseA,896,0,0,896,NaN,NaN,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1946.0,Unf,1.0,280.0,TA,TA,Y,0,0,0,0,0,0,NaN,MnPrv,NaN,0,4,2008,ConLD,79000


# Preprocessing Data before EDA

In [21]:
# histogram subplots
def subplot_histograms(dataframe, list_of_columns):
    nrows = int(np.ceil(len(list_of_columns)/4)) 
    fig, ax = plt.subplots(nrows=nrows, ncols=4,figsize=(15, nrows*3.5)) 
    ax = ax.ravel() 
    for i, column in enumerate(list_of_columns): 
        ax[i].hist(dataframe[column],bins=15)
        ax[i].set_title(f'{column} distribution',fontsize=14)
        ax[i].tick_params(labelsize=12)
        ax[i].set_xlabel(column, fontsize=14)
    plt.tight_layout()
    
# scatterplot subplots
def subplot_scatter(dataframe, list_of_columns):
    nrows = int(np.ceil(len(list_of_columns)/4)) 
    fig, ax = plt.subplots(nrows=nrows, ncols=4,figsize=(15, nrows*3.5)) 
    ax = ax.ravel() 
    for i, column in enumerate(list_of_columns): 
        sns.regplot(y=dataframe.saleprice, x=dataframe[column],ax=ax[i], \
                    scatter_kws={'facecolors':'skyblue','edgecolor':'skyblue'},
                    line_kws = {'color':'grey'})
        ax[i].set_title(f'{column} vs saleprice',fontsize=14)  
        ax[i].tick_params(labelsize=12)
        ax[i].set_xlabel(column, fontsize=14)
        ax[i].set_ylabel('saleprice', fontsize=14)
    plt.tight_layout()
    
# boxplot subplots
def subplot_box(dataframe, list_of_columns):
    nrows = int(np.ceil(len(list_of_columns)/4)) 
    fig, ax = plt.subplots(nrows=nrows, ncols=4,figsize=(15, nrows*3.5)) 
    ax = ax.ravel() 
    for i, column in enumerate(list_of_columns): 
        sns.boxplot(x = dataframe[column], y = dataframe.saleprice, width = 0.3, ax = ax[i], color='skyblue')
        ax[i].set_title(column,fontsize=14)  
        ax[i].tick_params(labelsize=12)
        ax[i].set_xlabel(column, fontsize=14)
#         ax[i].set_ylabel('saleprice', fontsize=14)
    plt.tight_layout()
    
# distribution plots (histogram, boxplot, probplot)
def dist_plots(df, list_of_columns):
    nrows = len(list_of_columns)
    fig, ax = plt.subplots(nrows = nrows, ncols = 3, figsize=(15, nrows*3.5))
    ax = ax.ravel()
    for i, col in enumerate(list_of_columns):
        sns.distplot(df[col], ax = ax[i*3-3], fit = stats.norm)
        ax[i*3-3].set_title(f'{col} distribution plot',fontsize=14)
        ax[i*3-3].tick_params(labelsize=12)
        ax[i*3-3].set_xlabel(col, fontsize=14)
        
        sns.boxplot(df[col], width = 0.2, ax = ax[i*3-2])
        ax[i*3-2].set_title(f'{col} box plot',fontsize=14)
        ax[i*3-2].tick_params(labelsize=12)
        ax[i*3-2].set_xlabel(col, fontsize=14)
        
        stats.probplot(df[col], plot = ax[i*3-1])
        ax[i*3-1].set_title(f'{col} probability plot', fontsize=14)
        ax[i*3-1].tick_params(labelsize=12)
        ax[i*3-1].set_xlabel(col, fontsize=14)
    plt.tight_layout()
plt.show()

In [22]:
import warnings
warnings.filterwarnings('ignore')

sns.set_style('ticks')

pd.set_option('display.max_columns', None)


below conveys a summarisation of all EDa, which shall be included within the functionn displayed thereafter.

Within 26 columns which have null values; categorical columns with null values will be filled with 'None', and null values in numerical columns will be replaced with 0. For columns with 1-2 null values, the rows containing that value will be dropped.

From the dataset there are a portion of columns which are highly skewed towards a single value or had numerous zeros. among these columns, the following are not predicted to be important in terms of saleprice, exlusion occurs later.
- `alley` (93.2% null)
- `miscval` (97% zeros)
- `lowqualfinsf` (98% zeros)
- `street` (99.7% single value)
- `utilities` (99.9% single value)
- `condition2` (98.7% single value)
- `roofmatl` (98.7% single value)
- `heating` (98.4% single value)
- `centralair` (93.1% single value)
- `electrical` (91.1% single value)
- `paveddrive` (90.7 single value)
- `fence` (80.5% zeros)
- `saletype` (86.8% single value)
- `bsmthalfbath` (94% zeros)
- `bsmtfintype2` (85.3% zeros)
- `bsmtfinsf2` (88% zeros)
- `bsmtcond` (89.% single value)
- `extercond` (86.7% single value)
- `garagequal` (89.6% single value)

The presence of the following may possibly affect saleprice (e.g. a house has a pool may be signifiantly more expensive than one without), and will be converted to presence-absence columns (e.g. 'poolpresence'):
- `3ssnporch` (98.7% zeros)
- `screenporch` (91.2% zeros)
- `enclosedporch` (84% zeros) (the four different porch columns - `3ssnporch`, `enclosedporch`, `openporch`, `screenporch` - will be combined.)
- `poolarea` (99.6% zeros)
- `poolqc` (99.6% single value)

The following columns with large proportions of a single value shall not be modified, being the minority values may likely affect saleprice. 
- `functional` (93.4% single value)
- `garagecond` (91.1% single value)
- `condition1` (86.2% single value)
- `bldgtype` (82.9% single value)

The following columns will be modified:
- `landslope` (95.2% single value) - to contain 2 categories, flat and sloped
- exterior materials - exterior2nd will be dropped
- `miscfeature` (96.8% single value) - to a shed-presence column
- `age_sold` will be added (difference between year sold and year built)
- `garageyrblt` will be removed
- `mssubclass` will be converted to categorical
- `remodelled`: yes/no (yearremod/add is the same as yearbuilt if there's no remodel)
- `garagepres`
- `fireplacepres`
- `bsmtpres`
- `masvnrpres`
- `2ndflrpres`
- `1stflrsf` will be deleted
- `2ndflrsf` will be converted to a second floor presence column
- houses with `grlivarea` bigger than the sum of 1st and 2nd floors will be placed into a new `bighouses` column

outliers will be dropped  within:
- `bsmtfinsf1`
- `grlivarea`
- `lotarea`
- `lotfrontage`

Finally, additonally collinear features will be dropped:
- `garagecars`
- `totrmsabvgrd`
- `totalbsmtsf`

## Preprocessing EDA

In [23]:
#format obtained from assistance upon codeMentor.com session, and codeacademy
# scatterplot subplots
def subplot_scatter(dataframe, list_of_columns):
    nrows = int(np.ceil(len(list_of_columns)/4)) 
    fig, ax = plt.subplots(nrows=nrows, ncols=4,figsize=(15, nrows*3.5)) 
    ax = ax.ravel() 
    for i, column in enumerate(list_of_columns): 
        sns.regplot(y=dataframe.saleprice, x=dataframe[column],ax=ax[i], \
                    scatter_kws={'facecolors':'skyblue','edgecolor':'skyblue'},
                    line_kws = {'color':'grey'})
        ax[i].set_title(f'{column} vs saleprice',fontsize=14)  
        ax[i].tick_params(labelsize=12)
        ax[i].set_xlabel(column, fontsize=14)
        ax[i].set_ylabel('saleprice', fontsize=14)
    plt.tight_layout()

# histogram subplots
def subplot_histograms(dataframe, list_of_columns):
    nrows = int(np.ceil(len(list_of_columns)/4)) 
    fig, ax = plt.subplots(nrows=nrows, ncols=4,figsize=(15, nrows*3.5)) 
    ax = ax.ravel() 
    for i, column in enumerate(list_of_columns): 
        ax[i].hist(dataframe[column],bins=15)
        ax[i].set_title(f'{column} distribution',fontsize=14)
        ax[i].tick_params(labelsize=12)
        ax[i].set_xlabel(column, fontsize=14)
    plt.tight_layout()
    
# distribution plots (histogram, boxplot, probplot)
def dist_plots(df, list_of_columns):
    nrows = len(list_of_columns)
    fig, ax = plt.subplots(nrows = nrows, ncols = 3, figsize=(15, nrows*3.5))
    ax = ax.ravel()
    for i, col in enumerate(list_of_columns):
        sns.distplot(df[col], ax = ax[i*3-3])
        ax[i*3-3].set_title(f'{col} distribution plot',fontsize=14)
        ax[i*3-3].tick_params(labelsize=12)
        ax[i*3-3].set_xlabel(col, fontsize=14)
        
        sns.boxplot(df[col], width = 0.2, ax = ax[i*3-2])
        ax[i*3-2].set_title(f'{col} box plot',fontsize=14)
        ax[i*3-2].tick_params(labelsize=12)
        ax[i*3-2].set_xlabel(col, fontsize=14)
        
        stats.probplot(df[col], plot = ax[i*3-1])
        ax[i*3-1].set_title(f'{col} probability plot', fontsize=14)
        ax[i*3-1].tick_params(labelsize=12)
        ax[i*3-1].set_xlabel(col, fontsize=14)
    plt.tight_layout()

In [24]:
#continued assistance from codementor.com
# import train
train = pd.read_csv ("~/desktop/DsI/submissions/projects/project_2/datasets/train.csv")

# run preprocessing function
train_preprocessed = preprocess_train(train)

# check df
print(train_preprocessed.columns)
train_preprocessed.info()

# no null values
# on the training data, the preprocessing function retains 2042 out of 2051 samples (99.5%)

Index(['index', 'id', 'mssubclass', 'mszoning', 'lotfrontage', 'lotarea',
       'lotshape', 'landcontour', 'lotconfig', 'landslope', 'neighborhood',
       'condition1', 'bldgtype', 'housestyle', 'overallqual', 'overallcond',
       'yearbuilt', 'roofstyle', 'masvnrtype', 'masvnrarea', 'exterqual',
       'foundation', 'bsmtqual', 'bsmtexposure', 'bsmtfintype1', 'bsmtfinsf1',
       'bsmtunfsf', 'heatingqc', 'grlivarea', 'bsmtfullbath', 'fullbath',
       'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'kitchenqual', 'functional',
       'fireplaces', 'fireplacequ', 'garagetype', 'garagefinish', 'garagearea',
       'garagecond', 'wooddecksf', 'mosold', 'yrsold', 'saleprice',
       'porchpres', 'poolpres', 'garagepres', 'fireplacepres', 'masvnrpres',
       'bsmtpres', 'shedpres', 'exterior', 'age_sold', 'remodelled',
       '2ndfloorpres', 'bighouse'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2046 entries, 0 to 2045
Data columns (total 58 columns):
 #   Co

## Relationship between saleprice & outliers
Below within the scatterplots, one observes a relatively linear relationship between saleprice & several of the features. For instance; `overallqual`, `yearbuilt`, `totalbsmtsf`, `1stflrsf`, `grlivarea`, `totrmsabvgrd`, `fireplaces`, `garagecars`, and `garagearea`. These variables present area as an important stance for predicting `saleprice`.

There are also a few obvious outliers, in 
- `lotarea`
- `lotfrontage`
- `1stflrsf`
- `bsmtfinsf1`
- `grlivarea`

In [25]:
# list with only numerical columns
col_numerical = [col for col in train.columns if train[col].dtype != np.object]

# list with only categorical columns
col_categorical = [col for col in train.columns if col not in col_numerical]

# preliminary filling of null values so the features can be visualised
train_copy = train.copy()
for col in col_numerical:
    train_copy[col] = train_copy[col].fillna(0.0)
for col in col_categorical:
    train_copy[col] = train_copy[col].fillna('None')
    
# plot distributions
dist_plots(train_copy,col_numerical=all)

TypeError: dist_plots() got an unexpected keyword argument 'col_numerical'

In [ ]:
# check scatterplots 
col_numerical = [col for col in train_preprocessed.columns if train_preprocessed[col].dtype != np.object]
subplot_scatter(train_preprocessed,col_numerical)

In [ ]:
#Histograms (categorical variables)
subplot_histograms(train_copy,col_categorical)

In [ ]:
#Boxplots (categorical variables)
# subplot_box(train_copy, col_categorical)

# Model Construction and Testing Parameters. 

In [ ]:
# #Load train data (Write train.csv directory) assistance from codementors.com
# test = pd.read_csv('~/desktop/DsI/submissions/projects/project_2/datasets/test.csv')
# train= pd.read_csv('~/desktop/DsI/submissions/projects/project_2/datasets/train.csv')

# data = train.append(test,sort=False) #Make train set and test set in the same data set

# data #Visualize the DataFrame data

In [ ]:
#Plot features with more than 1000 NULL values

features = []
nullValues = []
for i in data:
    if (data.isna().sum()[i])>1000 and i!='SalePrice':
        features.append(i)
        nullValues.append(data.isna().sum()[i])
y_pos = np.arange(len(features)) 
plt.bar(y_pos, nullValues, align='center', alpha=0.5)
plt.xticks(y_pos, features)
plt.ylabel('NULL Values')
plt.xlabel('Features')
plt.title('Features with more than 1000 NULL values')
plt.show()

In [ ]:
#Dealing with NULL values

data = data.dropna(axis=1, how='any', thresh = 1000) #Drop columns that contain more than 1000 NULL values
data = data.fillna(data.mean()) #Replace NULL values with mean values
data

In [ ]:
#Dealing with NULL values

data = pd.get_dummies(data) #Convert string values to integer values

In [ ]:
#Drop features that are correlated to each other

covarianceMatrix = data.corr()
listOfFeatures = [i for i in covarianceMatrix]
setOfDroppedFeatures = set() 
for i in range(len(listOfFeatures)) :
    for j in range(i+1,len(listOfFeatures)): #Avoid repetitions 
        feature1=listOfFeatures[i]
        feature2=listOfFeatures[j]
        if abs(covarianceMatrix[feature1][feature2]) > 0.8: #If the correlation between the features is > 0.8
            setOfDroppedFeatures.add(feature1) #Add one of them to the set
#I tried different values of threshold and 0.8 was the one that gave the best results

data = data.drop(setOfDroppedFeatures, axis=1)

In [ ]:
#Drop features that are not correlated with output

nonCorrelatedWithOutput = [column for column in data if abs(data[column].corr(data["SalePrice"])) < 0.045]
#I tried different values of threshold and 0.045 was the one that gave the best results

data = data.drop(nonCorrelatedWithOutput, axis=1)

In [ ]:
# #Plot one of the features with outliers

# plt.plot(data['LotArea'], data['SalePrice'], 'bo')
# plt.axvline(x=75000, color='r')
# plt.ylabel('SalePrice')
# plt.xlabel('LotArea')
# plt.title('SalePrice in function of LotArea')
# plt.show()

In [ ]:
#Seperating the data (Because removing outliers ⇔ removing rows, and we don't want to remove rows from test set)

newTrain = data.iloc[:1460]
newTest = data.iloc[1460:]

#Define a function that returns outlier values using percentile() method:

def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75]) #Get 1st and 3rd quartiles (25% -> 75% of data will be kept)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5) #Get lower bound
    upper_bound = quartile_3 + (iqr * 1.5) #Get upper bound
    return np.where((ys > upper_bound) | (ys < lower_bound)) #Get outlier values

#Third, we will drop the outlier values from the train set

trainWithoutOutliers = newTrain #We can't change train while running through it

for column in newTrain:
    outlierValuesList = np.ndarray.tolist(outliers_iqr(newTrain[column])[0]) #outliers_iqr() returns an array
    trainWithoutOutliers = newTrain.drop(outlierValuesList) #Drop outlier rows
    
trainWithoutOutliers = newTrain

In [ ]:
X = trainWithoutOutliers.drop("SalePrice", axis=1) #Remove SalePrice column
Y = np.log1p(trainWithoutOutliers["SalePrice"]) #Get SalePrice column {log1p(x) = log(x+1)}
reg = LinearRegression().fit(X, Y)

In [ ]:

#Make prediction

newTest = newTest.drop("SalePrice", axis=1) #Remove SalePrice column
pred = np.expm1(reg.predict(newTest))

#Submit prediction

sub = pd.DataFrame() #Create a new DataFrame for submission
sub['Id'] = test['Id']
sub['SalePrice'] = pred
sub.to_csv("submission.csv", index=False) #Convert DataFrame to .csv file

sub #Visualize the DataFrame sub